<a href="https://colab.research.google.com/github/MegumuTsukamoto/BERT_Stockmark/blob/main/Chapter_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\large{\text{5. 文章の穴埋め}}$

In [15]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


マスク付き言語モデルとしてのBERT...文章の1部のトークンを特殊トークン[MASK]に変換したものを入力として与え、[MASK]に入るものは何かを予測するタスクを用いて事前学習する。そのため、事前学習後のBERTでは一部が除かれた文章の穴埋めを行うことができる。

In [16]:
# 5-1
# PyTorchとMeCabはColaboratoryに最初からインストールされている
# !pip install transformers=4.5.0 fugashi==1.1.0 ipadic==1.0.0 ではエラー。下記でFugashiもIpadicも入る
!pip install transformers[ja]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
# 5-2
# ライブラリの読み込み
import numpy as np
import torch
# from transformers import BertJapanese-Tokenizer, BertForMaskedLM
from transformers import BertJapaneseTokenizer, BertForMaskedLM # 今回はMaskedLMを使う

In [18]:
# 5-3
# 東北大学の日本語モデルによるトークナイザとモデルのロード
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
bert_mlm = bert_mlm.cuda() # モデルをGPUに載せる

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
# 5-4
# 穴空き文章（MLMのインプット）のトークン化
text = '今日は[MASK]へ行く。'
tokenizer.tokenize(text)

['今日', 'は', '[MASK]', 'へ', '行く', '。']

In [20]:
# 5-5
# BertForMaskedLM は特殊トークン[MASK]に入るトークンを語彙の中から予測する。
# BertForMaskedLM の中にはBertModel と同様に、符号化された文章（トークン列）を入力する。
# 文章を符号化し、GPUに配置する
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.cuda()

# BERTに入力し、分類スコアを得る
# 系列長を考える必要がないので、単にinput_idsのみ入力
with torch.no_grad(): # 推論のみなので途中計算を保存しない
  output = bert_mlm(input_ids=input_ids)
  scores = output.logits

In [21]:
# 5-6
# 上記scoresから、[MASK]に入るトークンを予測

# ID列で、'[MASK]'の位置（IDの数は4）の位置を調べる（[CLS]:0番目からスタートして3番目）
input_ids = input_ids.cpu() # CPUに移して
input_ids = input_ids.numpy() # ndarrayに変換して
mask_position = input_ids[0].tolist().index(4) # ようやくリスト型にできる？
print(mask_position)

# スコアが最も良いトークンのIDを取り出し、トークンに変換する（今は、'東京'が最もスコアが高い）
id_best = scores[0, mask_position].argmax(-1).item()
token_best = tokenizer.convert_ids_to_tokens(id_best)
token_best = token_best.replace('##', '')

# 上で求めたトークンを[MASK]の部分と入れ替える
text = text.replace('[MASK]', token_best)
print(text)

3
今日は東京へ行く。


In [22]:
# 5-7
# [MASK]を、最もスコアが高い'東京'だけじゃなく、上位10個のトークンに置き換えた場合の処理を行う関数
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):
  
  # 文章中の最初の[MASK]をスコアの上位のトークンに置き換える
  # 上位何個を[MASK]の代わりに入れるかはnum_topk=で指定
  # 出力は穴埋めされた文章のリストと、置き換えられた
  
  # 文章を符号化し、BERTで分類スコアを得る
  input_ids = tokenizer.encode(text, return_tensors='pt')
  input_ids = input_ids.cuda()
  with torch.no_grad(): # 推論のみなので途中計算を保存しない
    output = bert_mlm(input_ids=input_ids)
    scores = output.logits
  
  # スコアが上位のトークンとそのスコアを求める
  input_ids = input_ids.cpu()
  input_ids = input_ids.numpy()
  mask_position = input_ids[0].tolist().index(4)
  topk = scores[0, mask_position].topk(num_topk)
  ids_topk = topk.indices # トークンのID
  tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk) # IDからトークンに変換
  scores_topk = topk.values.cpu().numpy() # スコア

  # 上で求めたトークンを[MASK]の部分と入れ替える
  text_topk = []
  for token in tokens_topk:
    token = token.replace('##', '')
    text_topk.append(text.replace('[MASK]', token, 1))

  return text_topk, scores_topk

text = '今日は[MASK]へ行く。'
text_topk, scores_topk = predict_mask_topk(text, tokenizer, bert_mlm, num_topk=10)
print(text_topk, sep='\n')
print(scores_topk, sep='\n')
print('-'*50)
print('アンパックした場合')
print('-'*50)
print(*text_topk, sep='\n')

['今日は東京へ行く。', '今日はハワイへ行く。', '今日は学校へ行く。', '今日はニューヨークへ行く。', '今日はどこへ行く。', '今日は空港へ行く。', '今日はアメリカへ行く。', '今日は病院へ行く。', '今日はそこへ行く。', '今日はロンドンへ行く。']
[9.178561  9.145953  8.923238  8.838815  8.319213  8.1805105 7.91759
 7.8333464 7.826694  7.807024 ]
--------------------------------------------------
アンパックした場合
--------------------------------------------------
今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。


（注）ここまでは、[MASK]が１つの場合の穴埋めを考えたが、以下のような２つある場合も考えることができる。
$$\text{今日は [MASK] [MASK] へ行く。}$$
1つの[MASK]に対して32,000通りの候補があるため、２つの場合は32,000×32,000通りの候補がある。このため、近似的な手法で「貪欲法」というものがある。これは、まず一番最初の[MASK]を最も高いスコアのトークンで穴埋めをし、その穴埋め後の文章から次の[MASK]の穴埋めを逐次的に行っていくという方法である。

In [23]:
# 5-8
# 貪欲法を使用して、文章の穴埋めを行う。
def greedy_prediction(text, tokenizer, bert_mlm):
  for _ in range(text.count('[MASK]')):
    text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
  return text

text = '今日は[MASK][MASK]へ行く。' # [MASK]が2つある文章
greedy_prediction(text, tokenizer, bert_mlm)

'今日は、東京へ行く。'

In [24]:
# 5-9
# 一方、BERTは文章を前から順番に生成していくのは得意でない
text = '今日は[MASK][MASK][MASK][MASK][MASK]' # [MASK]だらけの文章
greedy_prediction(text, tokenizer, bert_mlm) # デタラメな文章に

'今日は社会社会的な地位'

このように、大部分が[MASK]の場合、デタラメな文章が生成される。これはBERTの事前学習の段階で、文章のうちごく一部分のトークンのみを[MASK]に置き換えて、周りの文脈からトークンを予測するというタスクを用いているからであり、大部分が[MASK]の場合における予測は事前学習していない。文章を前から順番に生成するためには、事前学習の段階で現在までのトークンから次のトークンを予測するというタスクを用いる必要があり、GPTモデルはこのような方式で事前学習を行っている。

ビームサーチ...貪欲法では、前から順番にスコア最高のトークンで置き換えを行っているが、最終的な合計スコアの最適化になっているとは限らない。より良い近似手法がビームサーチで、これは複数文章を出力できる。これは、一番最初の[MASK]において、スコアが高い10個のトークンで穴埋めした10個の穴埋め後文章を作り、次の[MASK]においても、スコアが高い10個のトークンで穴埋めした10個の穴埋め後文章を作り、最終的に100個の文章から合計スコアが高い10個の文章をアウトプットとする。

In [25]:
# 5-10
# ビームサーチで文章の穴埋めを行う
def beam_search(text, tokenizer, bert_mlm, num_topk):
  
  num_mask = text.count('[MASK]')
  text_topk = [text]
  scores_topk = np.array([0])

  # 現在得られている、それぞれの文章に対して、最初の[MASK]をスコアが上位のトークンで穴埋めする
  for _ in range(num_mask):
    text_candidates = [] # それぞれの文章を穴埋めした結果を追加する
    score_candidates = [] # 穴埋めに使ったトークンのスコアを追加する

    for text_mask, score in zip(text_topk, scores_topk):
      text_topk_inner, scores_topk_inner = predict_mask_topk(
          text_mask, tokenizer, bert_mlm, num_topk
      )
      text_candidates.extend(text_topk_inner)
      score_candidates.append(score + scores_topk_inner)

    # 穴埋めによって生成された文章の中から合計スコアの高いものを選ぶ
    score_candidates = np.hstack(score_candidates) # 横方向に結合
    idx_list = score_candidates.argsort()[::-1][:num_topk]
    text_topk = [text_candidates[idx] for idx in idx_list]
    score_topk = score_candidates[idx_list]

  return text_topk

text = '今日は[MASK][MASK]へ行く。'
text_topk = beam_search(text, tokenizer, bert_mlm, num_topk=10)
print(text_topk, sep='\n')
print('-'*50)
print('アンパックした場合')
print('-'*50)
print(*text_topk, sep='\n') # 貪欲法と同じく自然な文章が出力され、バリエーションにも富んでいる

['今日は、東京へ行く。', '今日は、ハワイへ行く。', '今日は、学校へ行く。', '今日は、ニューヨークへ行く。', '今日は、空港へ行く。', '今日は、北海道へ行く。', '今日は、パリへ行く。', '今日は、アメリカへ行く。', '今日は、日本へ行く。', '今日は、病院へ行く。']
--------------------------------------------------
アンパックした場合
--------------------------------------------------
今日は、東京へ行く。
今日は、ハワイへ行く。
今日は、学校へ行く。
今日は、ニューヨークへ行く。
今日は、空港へ行く。
今日は、北海道へ行く。
今日は、パリへ行く。
今日は、アメリカへ行く。
今日は、日本へ行く。
今日は、病院へ行く。


In [26]:
# 5-11
# 貪欲法と同様に、大部分が[MASK]の文章では自然な文章の生成は難しい
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
text_topk = beam_search(text, tokenizer, bert_mlm, num_topk=10)
print(*text_topk, sep='\n')

今日は社会社会的な地位
今日は社会社会的な組織
今日は社会社会的なもの
今日は社会社会的な活動
今日は社会社会的な団体
今日は社会社会的な状況
今日は社会社会的な概念
今日は社会社会的な役割
今日は社会社会的な存在
今日は社会社会的な意味
